# Agent with tools
This notebook illustrates how to build a simple agent capable of using tools. 
We'll use crewAI to illustrate how agentics implements logical transduction

## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [1]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 94ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [2]:
from agentics import AG
from crewai.tools import tool
from ddgs import DDGS

## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """ Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    

2025-09-16 19:58:24.473 | DEBUG    | agentics.core.llm_connections:<module>:121 - AGENTICS is connecting to the following LLM API providers:
2025-09-16 19:58:24.474 | DEBUG    | agentics.core.llm_connections:<module>:124 - 0 - WatsonX
2025-09-16 19:58:24.474 | DEBUG    | agentics.core.llm_connections:<module>:129 - 1 - Gemini
2025-09-16 19:58:24.474 | DEBUG    | agentics.core.llm_connections:<module>:133 - 2 - OpenAI
2025-09-16 19:58:24.475 | DEBUG    | agentics.core.llm_connections:<module>:135 - Please add API keys in .env file to add or disconnect providers.
2025-09-16 19:58:24.480 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. Using the first available provider.
2025-09-16 19:58:24.480 | DEBUG    | agentics.core.llm_connections:get_llm_provider:31 - Available LLM providers: ['watsonx', 'gemini', 'openai']. Using 'watsonx'
2025-09-16 19:58:24.482 | DEBUG    | agentics.core.llm_connections:get_llm_provider:29 - No LLM provider specified. U

## Define an Agent

Use crew AI agent directly create a document searcher agent, who is instructed to use the web_search tool defined above. Note that reasoning steps and other parameters can be customized. 


In [3]:
from crewai import Agent
doc_agent = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=True,
    reasoning_steps=2,
    verbose=True,
    llm=AG.get_llm_provider()
)

## Define a Task

In [4]:
from crewai import Task

doc_task = Task(
    description="""Your task is to perform an extensive web search about
    the following question {question} and return a document providing answers to 
    the questions that explore several interesting aspects, each of them supported 
    by pertinent information from web search.  """,
    expected_output="""A markdown document with introduction and different sections""",
    agent=doc_agent,
    markdown=True
)

## Define a Crew

Crews are enviroments in which tasks are assigned to different agents following different policies. In this simple example we define a crew with a single agent and single task. 

In [5]:
from crewai import Crew

crew = Crew(
    agents=[doc_agent],
    tasks=[doc_task],
    verbose=True,
)

## Kickoff the crew

Crews are ultimatelly analoguous to very sophisticated function definitions, whose inputs are json dictonaries. In order to put them in action you should kickoff them providing input parameters.

In the Agent Trace stack you can see the plan made by the agent and the different execution steps which might involve tool calls.

In [6]:
result = crew.kickoff(inputs={"question": input("Ask your question> ") })

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7805cf0a-c514-4850-99f9-8c5c2b38b68c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🧠 Reasoning Plan ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ## Task Understanding and Planning                                                                             │
│                                                                                                                 │
│  As a Doc Searcher, my primary goal is to find answers to questions from the user using the available web       │
│  search tool. Given the task to perform an extensive web search about a specific question and return a          │
│  document providing answers, I will leverage my expertise in web search and report creation to deliver a        │
│  comprehensive markdown document.                                                                               │
│                                                                                                                 │
│  ### 1. Understanding of the Task                                                                               │
│                                                                                                                 │
│  From my professional perspective, this task involves:                                                          │
│  - Conducting a thorough web search on the given question.                                                      │
│  - Identifying and extracting pertinent information from the search results.                                    │
│  - Organizing the information into a structured markdown document with an introduction and separate sections    │
│  for different aspects of the question.                                                                         │
│  - Ensuring each section is supported by relevant data and insights found during the web search.                │
│                                                                                                                 │
│  ### 2. Key Steps to Completion                                                                                 │
│                                                                                                                 │
│  To complete this task, I will take the following key steps:                                                    │
│  1. **Clarify the Question**: Ensure I understand the question fully to focus the search effectively.           │
│  2. **Web Search**: Utilize the `fetch_data_async` tool to perform an extensive web search. This will involve   │
│  crafting relevant search queries to capture a broad range of information related to the question.              │
│  3. **Information Filtering and Organization**: Filter the search results to extract the most pertinent and     │
│  accurate information. Organize this information into logical sections within the markdown document.            │
│  4. **Content Creation**: Write clear, concise content for each section, ensuring that the information is not   │
│  only informative but also engaging and easy to understand.                                                     │
│  5. **Review and Edit**: Review the document for completeness, accuracy, and clarity. Edit as necessary to      │
│  ensure the document meets the expected standards.                                                              │
│                                                                                                                 │
│  ### 3. Approaching Challenges                                                                                  │
│                                                                                                                 │
│  Challenges might include:                                                                                      │
│  - **Information Overload**: Managing a large volume of

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Task: Your task is to perform an extensive web search about                                                    │
│      the following question  and return a document providing answers to                                         │
│      the questions that explore several interesting aspects, each of them supported                             │
│      by pertinent information from web search.                                                                  │
│                                                                                                                 │
│  Reasoning Plan:                                                                                                │
│  ## Task Understanding and Planning                                                                             │
│                                                                                                                 │
│  As a Doc Searcher, my primary goal is to find answers to questions from the user using the available web       │
│  search tool. Given the task to perform an extensive web search about a specific question and return a          │
│  document providing answers, I will leverage my expertise in web search and report creation to deliver a        │
│  comprehensive markdown document.                                                                               │
│                                                                                                                 │
│  ### 1. Understanding of the Task                                                                               │
│                                                                                                                 │
│  From my professional perspective, this task involves:                                                          │
│  - Conducting a thorough web search on the given question.                                                      │
│  - Identifying and extracting pertinent information from the search results.                                    │
│  - Organizing the information into a structured markdown document with an introduction and separate sections    │
│  for different aspects of the question.                                                                         │
│  - Ensuring each section is supported by relevant data and insights found during the web search.                │
│                                                                                                                 │
│  ### 2. Key Steps to Completion                                                                                 │
│                                                                                                                 │
│  To complete this task, I will take the following key steps:                                                    │
│  1. **Clarify the Question**: Ensure I understand the question fully to focus the search effectively.           │
│  2. **Web Search**: Utilize the `fetch_data_async` tool to perform an extensive web search. This will involve   │
│  crafting relevant search queries to capture a broad range of information related to the question.              │
│  3. **Information Filtering and Organization**: Filter the search results to extract the most pertinent and     │
│  accurate information. Organize this information into logical sections within the markdown document.            │
│  4. **Content Creation**: Write clear, concise content 

Output()

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'title': 'Wikipedia Search engine - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Search_engine',       │
│  'body': '2 days ago - There are two main types of search engine that have evolved: one is a system of          │
│  predefined and hierarchically ordered keywords that humans have programmed extensively . The other is a        │
│  system that generates an "inverted index" by analyzing texts it locates. This first form relies much more      │
│  heavily on the computer itself to do the bulk of the work. Most Web ...'}, {'title': 'Google In-Depth Guide    │
│  to How Google Search Works | Google Search Central | Documentation | Google for Developers', 'href':           │
│  'https://developers.google.com/search/docs/fundamentals/how-search-works', 'body': "Indexing: Google analyzes  │
│  the text, ... on Google, Google returns information that's relevant to the user's query. The first stage is    │
│  finding out what pages exist on the web ...."}, {'title': 'Google Support Do an Advanced Search on Google -    │
│  Computer - Google Search Help', 'href':                                                                        │
│  'https://support.google.com/websearch/answer/35890?hl=en&co=GENIE.Platform=Desktop', 'body': 'Under "Then      │
│  narrow your results by," choose the filters you want to use. You can add more than one filter. Click Advanced  │
│  Search.'}, {'title': 'MDN Web Docs Browsing the web - Learn web development | MDN', 'href':                    │
│  'https://developer.mozilla.org/en-US/docs/Learn_web_development/Getting_started/Environment_setup/Browsing_th  │
│  e_web', 'body': "By this point in the module, you should have multiple modern web browsers installed on your   │
│  computer or other available devices. This article goes deeper into using browsers, looking at how a web        │
│  browser works, the difference between some of the everyday things you'll interact with, and how ..."},         │
│  {'title': 'GCFGlobal Internet Basics: Using Search Engines', 'href':                                           │
│  'https://edu.gcfglobal.org/en/internetbasics/using-search-engines/1/', 'body': "There are many different       │
│  search engines you can use, but some of the most popular include Google, Yahoo!, and Bing. To perf...          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Extensive Web Search Guide                                                                                   │
│  This guide provides an overview of extensive web search techniques, including tips, tools, and strategies for  │
│  effective web searching.                                                                                       │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│  Extensive web search is a crucial skill in today's digital age. With the vast amount of information available  │
│  on the web, it's essential to know how to navigate and find relevant information efficiently.                  │
│                                                                                                                 │
│  ## Tips for Effective Web Search                                                                               │
│  Here are some tips to improve your web search skills:                                                          │
│  * **Use specific keywords**: Instead of using broad terms, try to use specific keywords related to your        │
│  search query.                                                                                                  │
│  * **Use quotes**: Grouping words together with quotes can help you find exact phrases and reduce irrelevant    │
│  results.                                                                                                       │
│  * **Use filters**: Most search engines offer filters to refine your search results by date, location, and      │
│  more.                                                                                                          │
│  * **Check multiple sources**: Verify information across multiple sources to ensure accuracy and reliability.   │
│                                                                                                                 │
│  ## Advanced Search Techniques                                                                                  │
│  For more advanced searches, you can use:                                                                       │
│  * **Boolean operators**: Use operators like AND, OR, and NOT to combine keywords and exclude irrelevant        │
│  results.                                                                                                       │
│  * **Site search**: Use the `site:` operator to search within a specific website or domain.                     │
│  * **Filetype search**: Use the `filetype:` operator to search for specific file types, such as PDFs or DOCs.   │
│                                                                                                                 │
│  ## Tools for Extensive Web Search                                                                              │
│  In addition to search engines, there are several tools available to help you perform extensive web searches:   │
│  * **Google Advanced Search**: Offers advanced search features, including filtering by date, location, and      │
│  more.                                                                                                          │
│  * **Google Scholar**: A search engine for scholarly li

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 20bae94a-c622-4b42-b078-456b9bd4cde6                                                                     │
│  Agent: Doc Searcher                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7805cf0a-c514-4850-99f9-8c5c2b38b68c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Extensive Web Search Guide                                                                     │
│  This guide provides an overview of extensive web search techniques, including tips, tools, and strategies for  │
│  effective web searching.                                                                                       │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│  Extensive web search is a crucial skill in today's digital age. With the vast amount of information available  │
│  on the web, it's essential to know how to navigate and find relevant information efficiently.                  │
│                                                                                                                 │
│  ## Tips for Effective Web Search                                                                               │
│  Here are some tips to improve your web search skills:                                                          │
│  * **Use specific keywords**: Instead of using broad terms, try to use specific keywords related to your        │
│  search query.                                                                                                  │
│  * **Use quotes**: Grouping words together with quotes can help you find exact phrases and reduce irrelevant    │
│  results.                                                                                                       │
│  * **Use filters**: Most search engines offer filters to refine your search results by date, location, and      │
│  more.                                                                                                          │
│  * **Check multiple sources**: Verify information across multiple sources to ensure accuracy and reliability.   │
│                                                                                                                 │
│  ## Advanced Search Techniques                                                                                  │
│  For more advanced searches, you can use:                                                                       │
│  * **Boolean operators**: Use operators like AND, OR, and NOT to combine keywords and exclude irrelevant        │
│  results.                                                                                                       │
│  * **Site search**: Use the `site:` operator to search within a specific website or domain.                     │
│  * **Filetype search**: Use the `filetype:` operator to search for specific file types, such as PDFs or DOCs.   │
│                                                                                                                 │
│  ## Tools for Extensive Web Search                                                                              │
│  In addition to search engines, there are several tools available to help you perform extensive web searches:   │
│  * **Google Advanced Search**: Offers advanced search features, including filtering by date, location, and      │
│  more.                                                

## Display the results as markdown

In [7]:
from IPython.display import Markdown, display

display(Markdown(result.raw))

# Extensive Web Search Guide
This guide provides an overview of extensive web search techniques, including tips, tools, and strategies for effective web searching.

## Introduction
Extensive web search is a crucial skill in today's digital age. With the vast amount of information available on the web, it's essential to know how to navigate and find relevant information efficiently.

## Tips for Effective Web Search
Here are some tips to improve your web search skills:
* **Use specific keywords**: Instead of using broad terms, try to use specific keywords related to your search query.
* **Use quotes**: Grouping words together with quotes can help you find exact phrases and reduce irrelevant results.
* **Use filters**: Most search engines offer filters to refine your search results by date, location, and more.
* **Check multiple sources**: Verify information across multiple sources to ensure accuracy and reliability.

## Advanced Search Techniques
For more advanced searches, you can use:
* **Boolean operators**: Use operators like AND, OR, and NOT to combine keywords and exclude irrelevant results.
* **Site search**: Use the `site:` operator to search within a specific website or domain.
* **Filetype search**: Use the `filetype:` operator to search for specific file types, such as PDFs or DOCs.

## Tools for Extensive Web Search
In addition to search engines, there are several tools available to help you perform extensive web searches:
* **Google Advanced Search**: Offers advanced search features, including filtering by date, location, and more.
* **Google Scholar**: A search engine for scholarly literature, including articles, theses, and books.
* **Web archives**: Services like the **Internet Archive** allow you to access historical versions of web pages and websites.

## Conclusion
Extensive web search is a valuable skill that can help you find relevant information efficiently. By using specific keywords, quotes, and filters, and leveraging advanced search techniques and tools, you can become a proficient web searcher. Remember to always verify information across multiple sources and use filters to refine your search results.